In [131]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import wget
import os
from datetime import datetime
from random import randrange, randint
from datetime import timedelta
import json

In [148]:
# Starting from the top
print('Starting crawling script.')

Starting crawling script.


In [149]:
# Get recipe categories
base = 'https://www.myfoodandfamily.com/recipes/'
pages = {'dinner': [3, 4, 6, 8, 7, 5],
        'regional': [412, 202, 118, 486, 372, 30, 362, 140, 149, 271],
        'holiday': [597, 10, 21],
        }

In [151]:
# want category, subcategory, and link
data = []

base = 'https://www.myfoodandfamily.com/recipes/'
for category in pages:
    subcategories = pages[category]
    for subcategoryid in subcategories:
        print('Scraping category: %s on id #%i' % (category, subcategoryid))
        url = base + str(subcategoryid)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        subcategoryname = soup.find('h1').text
        cards = soup.find_all('div', class_='cls_outerCard item-card')
        for card in cards:
            link = 'https://www.myfoodandfamily.com/' + card.find('a')['href']
            image = card.find('img')['data-yo-src']
            tup = (category, subcategoryname, link, image)
            data.append(tup)

Scraping category: dinner on id #3
Scraping category: dinner on id #4
Scraping category: dinner on id #6
Scraping category: dinner on id #8
Scraping category: dinner on id #7
Scraping category: dinner on id #5
Scraping category: regional on id #412
Scraping category: regional on id #202
Scraping category: regional on id #118
Scraping category: regional on id #486
Scraping category: regional on id #372
Scraping category: regional on id #30
Scraping category: regional on id #362
Scraping category: regional on id #140
Scraping category: regional on id #149
Scraping category: regional on id #271
Scraping category: holiday on id #597
Scraping category: holiday on id #10
Scraping category: holiday on id #21


In [152]:
print('There are %i recipes' % len(data))

There are 430 recipes


In [154]:
data[0]

('dinner',
 'Dinner Recipes',
 'https://www.myfoodandfamily.com//recipe/095157/baked-crab-rangoon?categoryid=3',
 'https://cdn-fsly.yottaa.net/5c8fa1e52bb0ac74cc828a3e/a954f3508a360137f0e97e3461d3e37f.90bd97202bb20137fdc3123dfe2baf36.yottaa.net/v~4b.179/adaptivemedia/rendition/95157.tif?id=c39fe037ca7739b4531cc9ba1c962e751c0d276d&ht=225&wd=225&version=1&clid=pim&yocs=1b_')

In [ ]:
totalData = []
for index, recipe in enumerate(data):
    if index % 20 == 0:
        print('On index %i' % index)
    category, subcategory, link, imageLink = recipe
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    title = soup.find('h1').text.strip()
    
    timeTag = soup.find('div', attrs = {'data-qa': 'qa-cook-timer'})
    if timeTag is not None:
        estimatedTime = timeTag.text.strip()
    else:
        estimatedTime = None
        
    descTag = soup.find('div', attrs = {'data-qa':'qa-prd-desc'})
    if descTag is not None:
        description = descTag.text.strip()
    else:
        description = None
    
    ingredients = []
    ingredientsList = soup.find_all('div', attrs = {'data-qa':'qa-ingreList'})
    for element in ingredientsList:
        ingredients.append(element.text.strip())
        
    procedure = []
    procedureContainer = soup.find('div', attrs = {'data-qa':'qa-recipe-make-details'})
    procedureList = procedureContainer.find_all('div', class_='krRecipeDirectionsDest')
    for step in procedureList:
        # [2:] to get rid of number
        procedure.append(step.text[2:].strip())
        
    # Don't download if already exists
    downloadLocation = f'./recipeImages/{title}.jpg'
    if not os.path.exists(downloadLocation):
        if imageLink.startswith('//'):
            imageLink = 'http:' + imageLink + '?'
            myfile = requests.get(imageLink)
            open(downloadLocation, 'wb').write(myfile.content)
        else:
            wget.download(imageLink, downloadLocation)
    imageName = title + '.jpg'
    
    recipeObject = {
        'category': category,
        'subcategory': subcategory,
        'title': title,
        'description': description,
        'ingredients': ingredients,
        'procedure': procedure,
        'imageName': imageName,
        'estimatedTime': estimatedTime
    }
    totalData.append(recipeObject)
        

On index 0
On index 20
On index 40
On index 60
On index 80
On index 100
On index 120
On index 140
On index 160
On index 180
On index 200
On index 220
On index 240
On index 260
On index 280
On index 300
On index 320
On index 340
On index 360


In [98]:
df = pd.DataFrame(totalData)

In [99]:
# Create random list of datetimes

In [101]:
df.head()

,category,description,imageName,ingredients,procedure,subcategory,title
0,dinner,Learn how to make these crispy Baked Crab Rang...,Baked Crab Rangoon.jpg,[4 oz. (1/2 of 8-oz. pkg.) PHILADELPHIA Neufch...,"[Heat oven to 350°F., Mix first 4 ingredients ...",Dinner Recipes,Baked Crab Rangoon
1,dinner,Enjoy a devilishly good Country-Style Smashed ...,Country-Style Smashed Potato Salad.jpg,"[3 lb. Yukon gold potatoes (about 6 large), pe...",[Cook potatoes in boiling water in saucepan 10...,Dinner Recipes,Country-Style Smashed Potato Salad
2,dinner,Enjoy a bowl of our Hearty Italian Sausage Sou...,Hearty Italian Sausage Soup.jpg,[1 pkg. (13 oz.) OSCAR MAYER Natural Uncured I...,[Cook sausage and onions in large skillet on m...,Dinner Recipes,Hearty Italian Sausage Soup
3,dinner,Savor Lentil Soup during cold months or year-r...,Lentil Soup.jpg,"[6 slices OSCAR MAYER Bacon, chopped, 1 small ...",[Cook bacon and onions in large saucepan 8 to ...,Dinner Recipes,Lentil Soup
4,dinner,Trying to find a speedy beef stir-fry that'll ...,Speedy Barbecue Beef Stir-Fry.jpg,"[1/4 cup KRAFT Original Barbecue Sauce, 3 Tbsp...",[Mix barbecue sauce and broth until blended; t...,Dinner Recipes,Speedy Barbecue Beef Stir-Fry


In [106]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [107]:
start = datetime.strptime('1/1/2018 12:00 AM', '%m/%d/%Y %I:%M %p')
end = datetime.strptime('11/1/2019 12:00 AM', '%m/%d/%Y %I:%M %p')

In [125]:
times = []
for i in range(len(df)):
    dates = []
    count = randint(1, 5)
    if count < 3:
        count = 1
    else:
        count -= 1
    for i in range(count):
        randomDate = random_date(start, end)
        randomDate = randomDate.strftime("%m/%d/%Y, %H:%M:%S")
        dates.append(randomDate)
    dates.sort()
    times.append(dates)

In [126]:
df['times'] = times

In [143]:
# Do favorites
isFavorite = [True if randint(1,4) == 4 else False for i in range(len(df))]
df['isFavorite'] = isFavorite

In [144]:
# Save to csv
df.to_csv('data.csv', index = False)

In [145]:
df

,category,description,imageName,ingredients,procedure,subcategory,title,times,isFavorite
0,dinner,Learn how to make these crispy Baked Crab Rang...,Baked Crab Rangoon.jpg,[4 oz. (1/2 of 8-oz. pkg.) PHILADELPHIA Neufch...,"[Heat oven to 350°F., Mix first 4 ingredients ...",Dinner Recipes,Baked Crab Rangoon,"[03/28/2019, 18:41:56]",False
1,dinner,Enjoy a devilishly good Country-Style Smashed ...,Country-Style Smashed Potato Salad.jpg,"[3 lb. Yukon gold potatoes (about 6 large), pe...",[Cook potatoes in boiling water in saucepan 10...,Dinner Recipes,Country-Style Smashed Potato Salad,"[03/07/2019, 18:39:20, 04/06/2018, 01:55:53, 0...",False
2,dinner,Enjoy a bowl of our Hearty Italian Sausage Sou...,Hearty Italian Sausage Soup.jpg,[1 pkg. (13 oz.) OSCAR MAYER Natural Uncured I...,[Cook sausage and onions in large skillet on m...,Dinner Recipes,Hearty Italian Sausage Soup,"[03/25/2018, 13:26:12, 05/31/2018, 14:16:24]",False
3,dinner,Savor Lentil Soup during cold months or year-r...,Lentil Soup.jpg,"[6 slices OSCAR MAYER Bacon, chopped, 1 small ...",[Cook bacon and onions in large saucepan 8 to ...,Dinner Recipes,Lentil Soup,"[01/25/2018, 07:53:07]",False
4,dinner,Trying to find a speedy beef stir-fry that'll ...,Speedy Barbecue Beef Stir-Fry.jpg,"[1/4 cup KRAFT Original Barbecue Sauce, 3 Tbsp...",[Mix barbecue sauce and broth until blended; t...,Dinner Recipes,Speedy Barbecue Beef Stir-Fry,"[04/21/2019, 09:56:15]",False
5,dinner,Escape from the ordinary: Mandarin oranges and...,Fruit & Almond Salad.jpg,"[1 pkg. (5 oz.) torn salad greens, 1 can (11 o...",[Combine all ingredients except dressing in la...,Dinner Recipes,Fruit & Almond Salad,"[01/04/2018, 00:47:05, 03/16/2019, 03:07:45, 0...",False
6,dinner,Do you know what totchos are? You will after w...,Loaded Totchos.jpg,"[1 pkg. (32 oz.) ORE-IDA TATER TOTS, 1 contain...","[Heat oven to 425ºF., Bake TATER TOTS as direc...",Dinner Recipes,Loaded Totchos,"[06/30/2018, 04:45:33]",False
7,dinner,This winning combination of peanuts and dressi...,Chicken in Cilantro-Peanut Sauce.jpg,"[1/2 cup chicken broth, 1/2 cup KRAFT Zesty It...","[Blend broth, 1/4 cup dressing, cilantro, nuts...",Dinner Recipes,Chicken in Cilantro-Peanut Sauce,"[02/28/2019, 07:27:06]",False
8,dinner,Who says a smart salad has to be mostly lettuc...,Grilled Garden Salad.jpg,"[2 ears corn on the cob, husks and silk remove...","[Heat grill to medium-high heat., Grill corn 1...",Dinner Recipes,Grilled Garden Salad,"[08/01/2018, 14:58:06, 09/20/2019, 16:07:58, 1...",False
9,dinner,Get some seafood into the cookout mix with Spi...,Spicy BBQ Shrimp Kabobs.jpg,"[1/2 cup KRAFT Original Barbecue Sauce, 10 arb...","[Heat grill to medium-high heat., Blend barbec...",Dinner Recipes,Spicy BBQ Shrimp Kabobs,"[01/04/2018, 06:00:02, 11/05/2018, 12:51:33]",True


In [146]:
# Change to json
jsonData = df.to_json(orient = 'records')

In [147]:
with open('jsonData.json', 'w') as json_file:
    json.dump(jsonData, json_file)